In [1]:
import boto3
import pandas as pd
import time

# Initialize Athena client
athena_client = boto3.client('athena')

# Set your S3 output location where query results will be stored
s3_output = 's3://jaath-buckets-0491f6b4-2be4-4ab9-9aa2-c62891ad4a9c/athena/'

# Your query
query = """
SELECT
    pickup_hour,
    pickup_location_id,
    rides,
    month
FROM glue_transformed
WHERE pickup_hour BETWEEN
    '2023-01-01 00:00:00' AND
    '2023-01-31 23:59:59'
ORDER BY pickup_hour;
"""

# Start the query execution
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={
        'Database': 'taxi_db_2324'
    },
    ResultConfiguration={
        'OutputLocation': s3_output,
    }
)

# Get the query execution ID
query_execution_id = response['QueryExecutionId']

# Wait for the query to complete
state = 'RUNNING'
while state in ['RUNNING', 'QUEUED']:
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    state = response['QueryExecution']['Status']['State']
    if state in ['RUNNING', 'QUEUED']:
        time.sleep(1)

# If query executed successfully, fetch the results
if state == 'SUCCEEDED':
    # Get the results
    results = athena_client.get_query_results(QueryExecutionId=query_execution_id)

    # Extract column names
    columns = [col['Label'] for col in results['ResultSet']['ResultSetMetadata']['ColumnInfo']]

    # Extract data rows
    rows = []
    for row in results['ResultSet']['Rows'][1:]:  # Skip the header row
        data = [field.get('VarCharValue', '') for field in row['Data']]
        rows.append(data)

    # Create pandas DataFrame
    df = pd.DataFrame(rows, columns=columns)

    # Convert data types as needed
    df['pickup_hour'] = pd.to_datetime(df['pickup_hour'])
    df['pickup_location_id'] = df['pickup_location_id'].astype(int)
    df['rides'] = df['rides'].astype(int)

    print(f"Query results loaded into DataFrame with {len(df)} rows")
    print(df.head())
else:
    print(f"Query failed with state: {state}")
    print(response['QueryExecution']['Status']['StateChangeReason'])

Query results loaded into DataFrame with 999 rows
  pickup_hour  pickup_location_id  rides month
0  2023-01-01                 213      0    01
1  2023-01-01                  42      2    01
2  2023-01-01                 212      0    01
3  2023-01-01                 210      0    01
4  2023-01-01                   5      0    01
